# Two View Classifier
This notebook was created to run on Kaggle using a TPU.

The model created uses a One View Classifier Model as a starting point.

Datasets used:
- CBIS-DDSM Two Views in TFRecords format: https://www.kaggle.com/dsv/4433185 (DOI: 10.34740/kaggle/dsv/4433185)
	
Change HEIGHT and WIDTH variables to use different resolutions and use One View Classifier trained in same resolution.

In [1]:
# IMPORTS
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os
import zipfile
import numpy as np
import tensorflow as tf
import math
import cv2

from tensorflow.keras.applications import EfficientNetB0
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from glob import glob

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import gc

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
print("Rodei imports")

2022-12-11 16:47:59.260740: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-12-11 16:47:59.260920: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Rodei imports


In [ ]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
# Simple test TPU
# Step 1: Get the credential from the Cloud SDK
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
user_credential = user_secrets.get_gcloud_credential()
print("Obtendo credentials")
# Step 2: Set the credentials
user_secrets.set_tensorflow_credential(user_credential)
print("Definindo credentials")

# Step 3: Use a familiar call to get the GCS path of the dataset
from kaggle_datasets import KaggleDatasets
#!ls /kaggle/input
GCS_DS_PATH = KaggleDatasets().get_gcs_path('two-views-cbis-ddsm-v2-tfrecords')
print("GCS_DS_PATH:")
print(GCS_DS_PATH)
VALID_FILENAMES = tf.io.gfile.glob(GCS_DS_PATH+'/**')
print(VALID_FILENAMES)

In [ ]:
#Read from bucket
print("DEBUG: "+ GCS_DS_PATH)
# ../input/cbisddsmpatchesv2tfrecordspng/data_patches_s10_v2_tfrecords
files = tf.io.gfile.glob(GCS_DS_PATH+'/data_tv_cbis_v2_tfrecords/test/**')
print(len(files))

# Get datasets

In [ ]:
BATCH_SIZE = 3 * strategy.num_replicas_in_sync
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_DS_PATH+'/data_tv_cbis_v2_tfrecords/train/train_??-??.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_DS_PATH+'/data_tv_cbis_v2_tfrecords/val/val_??-?.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_DS_PATH+'/data_tv_cbis_v2_tfrecords/test/test_??-??.tfrec')
AUTO = tf.data.experimental.AUTOTUNE
# IMAGE_SIZE = [1152, 896]
IMAGE_SIZE = [2304, 1792]
print("BATCH_SIZE, AUTO and IMAGE_SIZE defined")

In [ ]:
# Helper methods
#Ler datasets em tfrecord
def decode_image(image_data):
#     HEIGHT = 1152
#     WIDTH = 896
    HEIGHT = 2304
    WIDTH = 1792
    # DECODIFICAR A IMAGEM
    # Get 16 bit gray scalar tf.io.parse_tensor
    image = tf.image.decode_png(image_data, channels=1, dtype=tf.uint16)
    image = tf.image.resize(image, [HEIGHT, WIDTH])
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [HEIGHT,WIDTH,1])
    return image


def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image_cc": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "image_mlo": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "label": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image_cc = decode_image(example['image_cc'])
    image_mlo = decode_image(example['image_mlo'])
    label = tf.cast(example['label'], tf.int32)
    return image_cc, image_mlo, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image_cc": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "image_mlo": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "label": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image_cc = decode_image(example['image_cc'])
    image_mlo = decode_image(example['image_mlo'])
    idnum = example['label']
    return image_cc, image_mlo, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset
def data_augment_tv(image_cc, image_mlo, label):
    # Thanks to the dataset.prefetch(AUTO)
    # statement in the next function (below), this happens essentially
    # for free on TPU. Data pipeline code is executed on the "CPU"
    # part of the TPU while the TPU itself is computing gradients.
    image_cc = tf.image.random_flip_left_right(image_cc)
    image_cc = tf.image.random_flip_up_down(image_cc)
    image_mlo = tf.image.random_flip_left_right(image_mlo)
    image_mlo = tf.image.random_flip_up_down(image_mlo)
    return image_cc, image_mlo, label   

def subtract_mean(image):
    TRAINING_MEAN = 13729
    TRAINING_VAR = 69225734.9022153
    image = image - TRAINING_MEAN
    return image

def one_to_three_channels(image):
    image = tf.repeat(image, repeats=3, axis=2)
    return image

def normalize_for_model(image):
    SCALE = 257.0 #pixels [0.0, 255.0]
    return image/SCALE

def adjust_for_model_input_tv(image_cc, image_mlo, label):
    image_cc = subtract_mean(image_cc)
    image_cc = normalize_for_model(image_cc)
    image_cc = one_to_three_channels(image_cc)
    image_mlo = subtract_mean(image_mlo)
    image_mlo = normalize_for_model(image_mlo)
    image_mlo = one_to_three_channels(image_mlo)
    return (image_cc, image_mlo), label

def get_training_dataset():
    print("CREATING TRAINING DATASET")
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    print("Data augmentation")
    dataset = dataset.map(data_augment_tv, num_parallel_calls=AUTO)
    dataset = dataset.map(adjust_for_model_input_tv, num_parallel_calls=AUTO)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.map(adjust_for_model_input_tv, num_parallel_calls=AUTO)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_test_dataset(ordered=True):
    dataset = load_dataset(TEST_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.map(adjust_for_model_input_tv, num_parallel_calls=AUTO)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset
print("Helper methods to load datasets ready")

In [ ]:
# Ler datasets
ds_train = get_training_dataset()
ds_valid = get_validation_dataset()
print("Datasets de treino e validação lidos")

In [ ]:
import re
import matplotlib.pyplot as plt
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)
print("Training data shapes:")
for (image_cc, image_mlo), label in ds_train.take(3):
    print(image_cc.numpy().shape,image_mlo.numpy().shape, label.numpy().shape)
print("###################################")
NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
size_test = NUM_TEST_IMAGES
print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

# Train model

In [ ]:
#Segunda implementação
#mbblock
# implemantation from https://github.com/qubvel/efficientnet/blob/master/efficientnet/model.py
from tensorflow.keras import backend as backend
from tensorflow.keras import Model
CONV_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 2.0,
        'mode': 'fan_out',
        # EfficientNet actually uses an untruncated normal distribution for
        # initializing conv layers, but keras.initializers.VarianceScaling use
        # a truncated distribution.
        # We decided against a custom initializer for better serializability.
        'distribution': 'normal'
    }
}

def swish(x):
    """Swish activation function.
    # Arguments
        x: Input tensor.
    # Returns
        The Swish activation: `x * sigmoid(x)`.
    # References
        [Searching for Activation Functions](https://arxiv.org/abs/1710.05941)
    """
    if backend.backend() == 'tensorflow':
        try:
            # The native TF implementation has a more
            # memory-efficient gradient implementation
            return backend.tf.nn.swish(x)
        except AttributeError:
            pass

    return x * backend.sigmoid(x)

    
def mb_block(inputs, activation_fn=swish, drop_rate=0., name='',
          filters_in=2560, filters_out=2560, kernel_size=3, strides=2,
          expand_ratio=2, se_ratio=0.25, id_skip=True):
    """A mobile inverted residual block.
    # Arguments
        inputs: input tensor.
        activation_fn: activation function.
        drop_rate: float between 0 and 1, fraction of the input units to drop.
        name: string, block label.
        filters_in: integer, the number of input filters.
        filters_out: integer, the number of output filters.
        kernel_size: integer, the dimension of the convolution window.
        strides: integer, the stride of the convolution.
        expand_ratio: integer, scaling coefficient for the input filters.
        se_ratio: float between 0 and 1, fraction to squeeze the input filters.
        id_skip: boolean.
    # Returns
        output tensor for the block.
    """
    bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1

    # Expansion phase
    filters = filters_in * expand_ratio
    if expand_ratio != 1:
        x = layers.Conv2D(filters, 1,
                          padding='same',
                          use_bias=False,
                          kernel_initializer=CONV_KERNEL_INITIALIZER,
                          name=name + 'expand_conv')(inputs)
        x = layers.BatchNormalization(axis=bn_axis, name=name + 'expand_bn')(x)
        x = layers.Activation(activation_fn, name=name + 'expand_activation')(x)
    else:
        x = inputs

    # Depthwise Convolution
    x = layers.DepthwiseConv2D(kernel_size,
                               strides=strides,
                               padding='same',
                               use_bias=False,
                               depthwise_initializer=CONV_KERNEL_INITIALIZER,
                               name=name + 'dwconv')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=name + 'bn')(x)
    x = layers.Activation(activation_fn, name=name + 'activation')(x)

    # Squeeze and Excitation phase
    if 0 < se_ratio <= 1:
        filters_se = max(1, int(filters_in * se_ratio))
        target_shape = (1, 1, filters) if backend.image_data_format() == 'channels_last' else (filters, 1, 1)
        se = layers.GlobalAveragePooling2D(name=name + 'se_squeeze')(x)
        se = layers.Reshape(target_shape, name=name + 'se_reshape')(se)
        se = layers.Conv2D(filters_se, 1,
                           padding='same',
                           activation=activation_fn,
                           kernel_initializer=CONV_KERNEL_INITIALIZER,
                           use_bias=True,
                           name=name + 'se_reduce')(se)
        se = layers.Conv2D(filters, 1,
                           padding='same',
                           activation='sigmoid',
                           use_bias=True,
                           kernel_initializer=CONV_KERNEL_INITIALIZER,
                           name=name + 'se_expand')(se)
        if backend.backend() == 'theano':
            # For the Theano backend, we have to explicitly make
            # the excitation weights broadcastable.
            se = layers.Lambda(
                lambda x: backend.pattern_broadcast(x, [True, True, True, False]),
                output_shape=lambda input_shape: input_shape,
                name=name + 'se_broadcast')(se)
        x = layers.multiply([x, se], name=name + 'se_excite')

    # Output phase
    x = layers.Conv2D(filters_out, 1,
                      padding='same',
                      use_bias=False,
                      kernel_initializer=CONV_KERNEL_INITIALIZER,
                      name=name + 'project_conv')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=name + 'project_bn')(x)
    if (id_skip is True and strides == 1 and filters_in == filters_out):
        if drop_rate > 0:
            x = layers.Dropout(drop_rate,
                               noise_shape=(None, 1, 1, 1),
                               name=name + 'drop')(x)
        x = layers.add([x, inputs], name=name + 'add')

    return x

print("Blockv2 ready")

In [ ]:
from tensorflow.keras.models import load_model
def get_full_view_clf(file, input_layer, name):
    print("Getting file")
    full_view_model = load_model(file)    
    effb0 = full_view_model.layers[1]
    effb0._name = name
    effb0.trainable = False
    for layer in effb0.layers:
        layer._handle_name = layer._name+'_'+name
    print("Nome")
    print(effb0.name)
    x = effb0(input_layer)
    return x

def get_model(file):
    HEIGHT = 2304
    WIDTH = 1792
#     HEIGHT = 1152
#     WIDTH = 896
    input_CC = tf.keras.Input(shape=(HEIGHT,WIDTH,3), name='CC_view')
    input_MLO = tf.keras.Input(shape=(HEIGHT,WIDTH,3), name='MLO_view')
    fv_CC = get_full_view_clf(file,input_CC,'input_CC')
    fv_MLO = get_full_view_clf(file,input_MLO,'input_MLO')
    x = tf.keras.layers.Concatenate(axis=-1)([fv_CC, fv_MLO])
    x = mb_block(x, name='b1_')
    x = mb_block(x, name='b2_')
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(2, activation='softmax')(x)
    model = Model((input_CC, input_MLO), outputs=x)
    for i in range(len(model.weights)):
        model.weights[i]._handle_name = model.weights[i].name + "_" + str(i)
    print("Model ready to use")
    for layer in model.layers:
        layer.trainable = False
    model.layers[-1].trainable = True
    return model

print("Defining model")

In [ ]:
with strategy.scope():
    # Build model
    weights_file = '/kaggle/input/fullviewclassifiertpumodels/full_view_classifier_tpu_2304_1792_30ep_auc_081_54.h5'
    model = get_model(weights_file)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4) # Adam LR=1e-4
    metrics = ['sparse_categorical_accuracy']
    
    model.compile(
        optimizer = optimizer,
        loss = 'sparse_categorical_crossentropy',#CrossEntropyLoss
        metrics = metrics,
    )
model.summary()

In [ ]:
import sklearn.metrics
import gc
from collections import Counter
#from sklearn import metrics

def get_auc(y, pred):
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(y_true=y, y_score=pred)
    auc = sklearn.metrics.auc(fpr, tpr)
    print(auc)
    return

def acc_and_auc(model, ds_test):
    pred = model.evaluate(ds_test, verbose=0)
    print("Test Accuracy:", pred[1])
    print("Test Loss:", pred[0])
    print("Predicting")
    pred = model.predict(ds_test,verbose=0)
    #print(pred)
    print("Getting probabilities")
    use_pred = pred[:,1]
    #print(use_pred)
    print("Getting labels")
    ds = ds_test.unbatch()
    labels = np.asarray(list(ds.map(lambda x, y: y)))
    print("Obtaining AUC")
    get_auc(labels, use_pred)
    return

def data_visualization(matrix):
    num_cases = 0
    total_per_class = [0,0]
    for i in range(2):
        for j in range(2):
            num_cases = num_cases+matrix[i,j]
            total_per_class[i] = total_per_class[i] + matrix[i,j]
    for i in range(2):
        print("Classe: ",i)
        print("Elementos dessa classe: ", total_per_class[i])
        print("Acertos: ", matrix[i,i])
        print("% erro: ", (total_per_class[i]-matrix[i,i])/total_per_class[i])
        
def check_check_model_metrics(model, dataset, size):
    hits = 0
    last_acc_hit = 0
    dataset = dataset.unbatch().batch(BATCH_SIZE)
    batch = iter(dataset)
    NUM_BATCH = size//BATCH_SIZE
    cf_matrix = np.zeros([2,2])
    n_correct_batches = 0
    for i in range(NUM_BATCH):
        last_acc_hit = hits
        images, labels = next(batch)
        probabilities = model.predict(images)
        predictions = np.argmax(probabilities, axis=-1)
        labels_np = labels.numpy().astype(np.int64)
        print_batch = False
        for i in range(BATCH_SIZE):
            if(predictions[i] == labels_np[i]):
                hits = hits + 1
            else:
                print_batch = True
        if(print_batch):
            print(probabilities)
            print(predictions)
            print(labels_np)
            print(dict(Counter(predictions)))
            print((hits-last_acc_hit)/BATCH_SIZE)
        else:
            n_correct_batches = n_correct_batches + 1
        cf_batch_matrix = sklearn.metrics.confusion_matrix(labels_np, predictions, labels=[0,1])
        cf_matrix = cf_matrix + cf_batch_matrix
    print(probabilities)
    print("Correct batches")
    print(n_correct_batches)
    print("Accuracy")
    print(hits/(NUM_BATCH*BATCH_SIZE))
    print(cf_matrix)
    data_visualization(cf_matrix)
    import seaborn as sns
    import matplotlib.pyplot as plt

    ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='.4g')

    ax.set_title('Matriz de Confusão de Classificador de Patch\n\n');
    ax.set_xlabel('\nClasses Preditas')
    ax.set_ylabel('Classes Reais ');
    sns.set(font_scale=1.4)
    ## Ticket labels - List must be in alphabetical order
    ax.xaxis.set_ticklabels(['Benigno','Maligno'])
    ax.yaxis.set_ticklabels(['Benigno','Maligno'])

    ## Display the visualization of the Confusion Matrix.
    plt.show()
    
def show_history(history, epochs):
    # PLOT ACC TREINO E VAL
    print(history.history.keys())
    print("Calculating the accuracy")
    print(history.history.keys())
    print("Calculating the accuracy")
    acc = history.history['sparse_categorical_accuracy']
    val_acc = history.history['val_sparse_categorical_accuracy']
    print("Calculating the loss")
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(epochs)
    print("The results are being visualized")
    plt.figure(figsize=(20, 60))
    # Acuracia
    plt.subplot(3, 1, 1)
    plt.plot(epochs_range, acc, label='Treino')
    plt.plot(epochs_range, val_acc, label='Validação')
    plt.legend(loc='lower right')
    plt.title('Acurácia de treino e validação')
    #Loss
    plt.subplot(3, 1, 2)
    plt.plot(epochs_range, loss, label='Treino')
    plt.plot(epochs_range, val_loss, label='Validação')
    plt.legend(loc='upper right')
    plt.title('Perda de treino e validação')
    plt.show()
    
print("Funções para obter métricas prontas")

In [ ]:
epochs = 20
batch_size = BATCH_SIZE
history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True,
    verbose=2,
)

In [ ]:
print("Verificando métricas treino apenas camada densa")
show_history(history,epochs)
ds_test = get_test_dataset()
size_test = NUM_TEST_IMAGES
check_check_model_metrics(model, ds_test, size_test)
acc_and_auc(model, ds_test)

In [ ]:
with strategy.scope():
    # Build model
    for i in range(4,33):
        model.layers[i].trainable = True
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6) # Adam LR=1e-6
    metrics = ['sparse_categorical_accuracy']
    
    model.compile(
        optimizer = optimizer,
        loss = 'sparse_categorical_crossentropy',#CrossEntropyLoss
        metrics = metrics,
    )
    model.summary()

In [ ]:
# Training
# Fit Model
epochs = 20
batch_size = BATCH_SIZE
history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True,
    verbose=2,
)

In [ ]:
print("Verificando métricas treino apenas camada densa + MBConvs")
show_history(history,epochs)
check_check_model_metrics(model, ds_test, size_test)
acc_and_auc(model, ds_test)

In [ ]:
# Save model
attemp = 0
for attemp in range(5):
    try:
        model.save("two_view_classifier_tpu_1152_896.h5")
        break
    except:
        print("Tentando pela vez numero ", attemp)
print("Model saved")

# Test model

In [ ]:
with strategy.scope():
    model_file = 'two_view_classifier_tpu_1152_896.h5'
    print("Getting file")
    test_model = load_model(model_file, compile=True)
ds_test = get_test_dataset()
print("Verificando métricas treino tudo")
show_history(history,epochs)
check_check_model_metrics(test_model, ds_test, size_test)
acc_and_auc(test_model, ds_test)

<a href="./two_view_cbis_v2_tpu_1152_res2.h5"> Download Model </a>